# Importing Packages

In [5]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision

In [6]:
from unet import build_unet
from customMetrics import dice_loss, dice_coef, iou

# Setting The Environment

In [3]:
H = 512
W = 512

In [7]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Hyperparameters """
batch_size = 1
lr = 1e-4
num_epochs = 100
model_path = os.path.join("files", "model.h5")
csv_path = os.path.join("files", "data.csv")

In [8]:
""" Dataset """
dataset_path = "new_data"
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "valid")

# Utils Functions

In [9]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [10]:
def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.jpg")))
    y = sorted(glob(os.path.join(path, "mask", "*.jpg")))
    return x, y

In [11]:
def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

In [12]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

In [13]:
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

In [14]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y


In [15]:
def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

# Loading The Dataset

In [16]:
train_x, train_y = load_data(train_path)
# train_x, train_y = shuffling(train_x, train_y)
valid_x, valid_y = load_data(valid_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")


Train: 100 - 100
Valid: 20 - 20


In [17]:
""" Directory to save files """
create_dir("files")

In [18]:
train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

train_steps = len(train_x)//batch_size
valid_setps = len(valid_x)//batch_size

if len(train_x) % batch_size != 0:
    train_steps += 1
if len(valid_x) % batch_size != 0:
    valid_setps += 1

# The Model

In [19]:
""" Model """
model = build_unet((H, W, 3))
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 512, 512, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 512, 512, 64) 0           batch_normalization[0][0]        
_______________________________________________________________________________________________

## Callbacks

In [20]:
callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
    CSVLogger(csv_path),
    TensorBoard(),
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
]

In [21]:
model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    steps_per_epoch=train_steps,
    validation_steps=valid_setps,
    callbacks=callbacks
)

Epoch 1/100
100/100 [==============================] - 119s 678ms/step - loss: 0.5716 - dice_coef: 0.4284 - iou: 0.2784 - recall: 0.5151 - precision: 0.5576 - val_loss: 0.8853 - val_dice_coef: 0.1147 - val_iou: 0.0609 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00

Epoch 00001: val_loss improved from inf to 0.88532, saving model to files\model.h5
Epoch 2/100
100/100 [==============================] - 65s 650ms/step - loss: 0.4398 - dice_coef: 0.5602 - iou: 0.3906 - recall: 0.4167 - precision: 0.8374 - val_loss: 0.9231 - val_dice_coef: 0.0769 - val_iou: 0.0400 - val_recall: 3.5947e-06 - val_precision: 0.0229

Epoch 00002: val_loss did not improve from 0.88532
Epoch 3/100
100/100 [==============================] - 62s 623ms/step - loss: 0.3937 - dice_coef: 0.6063 - iou: 0.4365 - recall: 0.4088 - precision: 0.8754 - val_loss: 0.9194 - val_dice_coef: 0.0806 - val_iou: 0.0423 - val_recall: 0.0121 - val_precision: 0.1838

Epoch 00003: val_loss did not improve from 0.88532
Epoch 4/100
1


Epoch 00025: val_loss improved from 0.28007 to 0.27751, saving model to files\model.h5
Epoch 26/100
100/100 [==============================] - 63s 631ms/step - loss: 0.1910 - dice_coef: 0.8090 - iou: 0.6798 - recall: 0.3998 - precision: 0.9747 - val_loss: 0.2676 - val_dice_coef: 0.7324 - val_iou: 0.5793 - val_recall: 0.3618 - val_precision: 0.9462

Epoch 00026: val_loss improved from 0.27751 to 0.26763, saving model to files\model.h5
Epoch 27/100
100/100 [==============================] - 62s 618ms/step - loss: 0.1882 - dice_coef: 0.8118 - iou: 0.6839 - recall: 0.4003 - precision: 0.9757 - val_loss: 0.2695 - val_dice_coef: 0.7305 - val_iou: 0.5769 - val_recall: 0.3588 - val_precision: 0.9451

Epoch 00027: val_loss did not improve from 0.26763
Epoch 28/100
100/100 [==============================] - 61s 614ms/step - loss: 0.1851 - dice_coef: 0.8149 - iou: 0.6882 - recall: 0.4011 - precision: 0.9770 - val_loss: 0.2886 - val_dice_coef: 0.7114 - val_iou: 0.5545 - val_recall: 0.3311 - val_p

In [26]:
def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)